In [1]:
import datetime
import numpy as np
import os, sys

from backtest import Backtest
from data import HistoricCSVDataHandler
from event import SignalEvent
from execution import SimulatedExecutionHandler
from portfolio import Portfolio
from strategy import Strategy

# Added by Brad on 20191006
from plots import PlotEquityCurve

# Added by Brad on 20191007
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# mac.py
print("Executing mac.py")
class MovingAverageCrossStrategy(Strategy):
    """
    Carries out a basic Moving Average Crossover strategy with a
    short/long simple weighted moving average. Default short/long
    windows are 12/26 periods respectively.
    """

    def __init__(self, bars, events, short_window=12, long_window=26):
        """
        Initialises the buy and hold strategy.

        Parameters:
        bars - The DataHandler object that provides bar information
        events - The Event Queue object.
        short_window - The short moving average lookback.
        long_window - The long moving average lookback.
        """
        self.bars = bars
        self.symbol_list = self.bars.symbol_list
        self.events = events
        self.short_window = short_window
        self.long_window = long_window

        # Set to True if a symbol is in the market
        self.bought = self._calculate_initial_bought()

    def _calculate_initial_bought(self):
        """
        Adds keys to the bought dictionary for all symbols
        and sets them to 'OUT'.
        """
        bought = {}
        for s in self.symbol_list:
            bought[s] = 'OUT'
        return bought

    def calculate_signals(self, event):
        """
        Generates a new set of signals based on the MAC
        SMA with the short window crossing the long window
        meaning a long entry and vice versa for a short entry.    

        Parameters
        event - A MarketEvent object. 
        """
        if event.type == 'MARKET':
            for symbol in self.symbol_list:
                bars = self.bars.get_latest_bars_values(symbol, "close", N=self.long_window)

                if bars is not None and bars != []:
                    short_sma = np.mean(bars[-self.short_window:])
                    long_sma = np.mean(bars[-self.long_window:])

                    dt = self.bars.get_latest_bar_datetime(symbol)
                    sig_dir = ""
                    strength = 1.0
                    strategy_id = 1

                    if short_sma > long_sma and self.bought[symbol] == "OUT":
                        sig_dir = 'LONG'
                        signal = SignalEvent(strategy_id, symbol, dt, sig_dir, strength)
                        self.events.put(signal)
                        self.bought[symbol] = 'LONG'

                    elif short_sma < long_sma and self.bought[symbol] == "LONG":
                        sig_dir = 'EXIT'
                        signal = SignalEvent(strategy_id, symbol, dt, sig_dir, strength)
                        self.events.put(signal)
                        self.bought[symbol] = 'OUT'


if __name__ == "__main__":
    # csv_dir = REPLACE_WITH_YOUR_CSV_DIR_HERE
    csv_dir = os.getcwd()
#     symbol_list = ['AAPL','AMZN','JNJ','V','XOM','WMT','VZ','BHP','NEE','SPG']
    symbol_list = ['jd000d']
    initial_capital = 1000000.0
    start_date = datetime.datetime(2015,4,1,0,0,0)
    heartbeat = 0.0

    backtest = Backtest(csv_dir, 
                        symbol_list, 
                        initial_capital, 
                        heartbeat,
                        start_date,
                        HistoricCSVDataHandler, 
                        SimulatedExecutionHandler, 
                        Portfolio, 
                        MovingAverageCrossStrategy)
    
    backtest.simulate_trading()

Executing backtest.py
Executing event.py
Executing data.py
Executing execution.py
Executing performance.py
Executing portfolio.py
Executing strategy.py
Executing mac.py
Creating DataHandler, Strategy, Portfolio and ExecutionHandler
Running Backtest...


/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Creating summary stats...
Creating equity curve...
                 cash  commission  jd000d      total  returns  equity_curve  \
datetime                                                                      
2018-12-18  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-19  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-20  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-21  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-24  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-25  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-26  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-27  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-28  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   
2018-12-28  1018340.2        59.8     0.0  1018340.2      0.0       1.01834   

